In [226]:
import numpy as np, pandas as pd, polars as pl
from enum import Enum
from sklearn import preprocessing as skp

# Import and clean data

In [227]:
train = pd.read_csv("train.csv").drop('id', axis=1)
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 750000 non-null  object 
 1   Episode_Title                750000 non-null  object 
 2   Episode_Length_minutes       662907 non-null  float64
 3   Genre                        750000 non-null  object 
 4   Host_Popularity_percentage   750000 non-null  float64
 5   Publication_Day              750000 non-null  object 
 6   Publication_Time             750000 non-null  object 
 7   Guest_Popularity_percentage  603970 non-null  float64
 8   Number_of_Ads                749999 non-null  float64
 9   Episode_Sentiment            750000 non-null  object 
 10  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), object(6)
memory usage: 62.9+ MB


,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031


In [228]:
test = pd.read_csv("test.csv").drop('id', axis=1)
test.info()
test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 250000 non-null  object 
 1   Episode_Title                250000 non-null  object 
 2   Episode_Length_minutes       221264 non-null  float64
 3   Genre                        250000 non-null  object 
 4   Host_Popularity_percentage   250000 non-null  float64
 5   Publication_Day              250000 non-null  object 
 6   Publication_Time             250000 non-null  object 
 7   Guest_Popularity_percentage  201168 non-null  float64
 8   Number_of_Ads                250000 non-null  float64
 9   Episode_Sentiment            250000 non-null  object 
dtypes: float64(4), object(6)
memory usage: 19.1+ MB


,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment
0,Educational Nuggets,Episode 73,78.96,Education,38.11,Saturday,Evening,53.33,1.0,Neutral
1,Sound Waves,Episode 23,27.87,Music,71.29,Sunday,Morning,NaN,0.0,Neutral
2,Joke Junction,Episode 11,69.10,Comedy,67.89,Friday,Evening,97.51,0.0,Positive
3,Comedy Corner,Episode 73,115.39,Comedy,23.40,Sunday,Morning,51.75,2.0,Positive
4,Life Lessons,Episode 50,72.32,Lifestyle,58.10,Wednesday,Morning,11.30,2.0,Neutral


### Split Categorical and Numerical Data

In [229]:
train_n = train.iloc[:,[2,4,7,8]]
train_c = train.iloc[:,[0,1,3,5,6,9]]
print(train_n.head())
train_c.head()

   Episode_Length_minutes  Host_Popularity_percentage  \
0                     NaN                       74.81   
1                  119.80                       66.95   
2                   73.90                       69.97   
3                   67.17                       57.22   
4                  110.51                       80.07   

   Guest_Popularity_percentage  Number_of_Ads  
0                          NaN            0.0  
1                        75.95            2.0  
2                         8.97            0.0  
3                        78.70            2.0  
4                        58.68            3.0  


,Podcast_Name,Episode_Title,Genre,Publication_Day,Publication_Time,Episode_Sentiment
0,Mystery Matters,Episode 98,True Crime,Thursday,Night,Positive
1,Joke Junction,Episode 26,Comedy,Saturday,Afternoon,Negative
2,Study Sessions,Episode 16,Education,Tuesday,Evening,Negative
3,Digital Digest,Episode 45,Technology,Monday,Morning,Positive
4,Mind & Body,Episode 86,Health,Monday,Afternoon,Neutral


### Categorical Data

#### Episode Number -> Int

In [230]:
train_c['Episode_Title'].unique()

array(['Episode 98', 'Episode 26', 'Episode 16', 'Episode 45',
       'Episode 86', 'Episode 19', 'Episode 47', 'Episode 44',
       'Episode 32', 'Episode 81', 'Episode 66', 'Episode 62',
       'Episode 76', 'Episode 37', 'Episode 20', 'Episode 82',
       'Episode 72', 'Episode 61', 'Episode 100', 'Episode 54',
       'Episode 17', 'Episode 36', 'Episode 97', 'Episode 27',
       'Episode 31', 'Episode 88', 'Episode 38', 'Episode 92',
       'Episode 74', 'Episode 30', 'Episode 63', 'Episode 67',
       'Episode 77', 'Episode 4', 'Episode 93', 'Episode 24', 'Episode 1',
       'Episode 2', 'Episode 25', 'Episode 56', 'Episode 75',
       'Episode 12', 'Episode 21', 'Episode 6', 'Episode 85',
       'Episode 23', 'Episode 33', 'Episode 7', 'Episode 53',
       'Episode 15', 'Episode 43', 'Episode 71', 'Episode 69',
       'Episode 13', 'Episode 89', 'Episode 3', 'Episode 64',
       'Episode 73', 'Episode 79', 'Episode 94', 'Episode 80',
       'Episode 42', 'Episode 10', 'Episode 48

In [231]:
def cutEpisode(title):
    return int(title[8:])

train_c['Episode_Title'] = train_c['Episode_Title'].apply(cutEpisode)
train_c['Episode_Title'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 750000 entries, 0 to 749999
Series name: Episode_Title
Non-Null Count   Dtype
--------------   -----
750000 non-null  int64
dtypes: int64(1)
memory usage: 5.7 MB


C:\Users\evan\AppData\Local\Temp\ipykernel_12964\1618039620.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_c['Episode_Title'] = train_c['Episode_Title'].apply(cutEpisode)


#### Genre, Publication Day and Time, Sentiment -> Enum

In [232]:
print(train_c['Genre'].unique(), train_c['Genre'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 750000 entries, 0 to 749999
Series name: Genre
Non-Null Count   Dtype 
--------------   ----- 
750000 non-null  object
dtypes: object(1)
memory usage: 5.7+ MB
['True Crime' 'Comedy' 'Education' 'Technology' 'Health' 'News' 'Music'
 'Sports' 'Business' 'Lifestyle'] None


In [233]:
genre_onehot = pd.get_dummies(train_c['Genre'])
#test = skp.OneHotEncoder()
#test.fit(train_c['Genre'])
#test.categories_

genre_onehot.head(100)

,Business,Comedy,Education,Health,Lifestyle,Music,News,Sports,Technology,True Crime
0,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0
4,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,1
96,0,0,0,0,1,0,0,0,0,0
97,0,0,0,0,0,0,1,0,0,0
98,0,0,0,0,1,0,0,0,0,0


In [234]:
train_c.drop(columns=['Genre'], inplace=True)

C:\Users\evan\AppData\Local\Temp\ipykernel_12964\3666887753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_c.drop(columns=['Genre'], inplace=True)


In [235]:
train_c = pd.concat([train_c, genre_onehot], axis=1) 
train_c.head()

,Podcast_Name,Episode_Title,Publication_Day,Publication_Time,Episode_Sentiment,Business,Comedy,Education,Health,Lifestyle,Music,News,Sports,Technology,True Crime
0,Mystery Matters,98,Thursday,Night,Positive,0,0,0,0,0,0,0,0,0,1
1,Joke Junction,26,Saturday,Afternoon,Negative,0,1,0,0,0,0,0,0,0,0
2,Study Sessions,16,Tuesday,Evening,Negative,0,0,1,0,0,0,0,0,0,0
3,Digital Digest,45,Monday,Morning,Positive,0,0,0,0,0,0,0,0,1,0
4,Mind & Body,86,Monday,Afternoon,Neutral,0,0,0,1,0,0,0,0,0,0


--------------------

In [236]:
print(train_c['Episode_Sentiment'].unique(), train_c['Episode_Sentiment'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 750000 entries, 0 to 749999
Series name: Episode_Sentiment
Non-Null Count   Dtype 
--------------   ----- 
750000 non-null  object
dtypes: object(1)
memory usage: 5.7+ MB
['Positive' 'Negative' 'Neutral'] None


In [237]:
def sentimentToInt(sentiment) -> int:
    match sentiment:
        case 'Positive': return 1
        case 'Negative': return 0
        case 'Neutral': return -1

In [238]:
train_c['Episode_Sentiment'] = train_c['Episode_Sentiment'].apply(sentimentToInt)
train_c.head()

,Podcast_Name,Episode_Title,Publication_Day,Publication_Time,Episode_Sentiment,Business,Comedy,Education,Health,Lifestyle,Music,News,Sports,Technology,True Crime
0,Mystery Matters,98,Thursday,Night,1,0,0,0,0,0,0,0,0,0,1
1,Joke Junction,26,Saturday,Afternoon,0,0,1,0,0,0,0,0,0,0,0
2,Study Sessions,16,Tuesday,Evening,0,0,0,1,0,0,0,0,0,0,0
3,Digital Digest,45,Monday,Morning,1,0,0,0,0,0,0,0,0,1,0
4,Mind & Body,86,Monday,Afternoon,-1,0,0,0,1,0,0,0,0,0,0


--------------------

In [239]:
print(train_c['Publication_Time'].unique(), train_c['Publication_Time'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 750000 entries, 0 to 749999
Series name: Publication_Time
Non-Null Count   Dtype 
--------------   ----- 
750000 non-null  object
dtypes: object(1)
memory usage: 5.7+ MB
['Night' 'Afternoon' 'Evening' 'Morning'] None


0 = Morning   
1 = Afternoon    
2 = Evening  
3 = Night  

In [240]:
def pubTimeToInt(dfTime):
    match dfTime:
        case 'Morning': return 0
        case 'Afternoon': return 1
        case 'Evening': return 2
        case 'Night': return 3

In [241]:
train_c['Publication_Time'] = train_c['Publication_Time'].apply(pubTimeToInt)
train_c.head()

,Podcast_Name,Episode_Title,Publication_Day,Publication_Time,Episode_Sentiment,Business,Comedy,Education,Health,Lifestyle,Music,News,Sports,Technology,True Crime
0,Mystery Matters,98,Thursday,3,1,0,0,0,0,0,0,0,0,0,1
1,Joke Junction,26,Saturday,1,0,0,1,0,0,0,0,0,0,0,0
2,Study Sessions,16,Tuesday,2,0,0,0,1,0,0,0,0,0,0,0
3,Digital Digest,45,Monday,0,1,0,0,0,0,0,0,0,0,1,0
4,Mind & Body,86,Monday,1,-1,0,0,0,1,0,0,0,0,0,0


---

In [242]:
print(train_c['Publication_Day'].unique(), train_c['Publication_Day'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 750000 entries, 0 to 749999
Series name: Publication_Day
Non-Null Count   Dtype 
--------------   ----- 
750000 non-null  object
dtypes: object(1)
memory usage: 5.7+ MB
['Thursday' 'Saturday' 'Tuesday' 'Monday' 'Sunday' 'Wednesday' 'Friday'] None


In [243]:
print(train_c['Podcast_Name'].unique(), train_c['Podcast_Name'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 750000 entries, 0 to 749999
Series name: Podcast_Name
Non-Null Count   Dtype 
--------------   ----- 
750000 non-null  object
dtypes: object(1)
memory usage: 5.7+ MB
['Mystery Matters' 'Joke Junction' 'Study Sessions' 'Digital Digest'
 'Mind & Body' 'Fitness First' 'Criminal Minds' 'News Roundup'
 'Daily Digest' 'Music Matters' 'Sports Central' 'Melody Mix' 'Game Day'
 'Gadget Geek' 'Global News' 'Tech Talks' 'Sport Spot' 'Funny Folks'
 'Sports Weekly' 'Business Briefs' 'Tech Trends' 'Innovators'
 'Health Hour' 'Comedy Corner' 'Sound Waves' 'Brain Boost'
 "Athlete's Arena" 'Wellness Wave' 'Style Guide' 'World Watch' 'Humor Hub'
 'Money Matters' 'Healthy Living' 'Home & Living' 'Educational Nuggets'
 'Market Masters' 'Learning Lab' 'Lifestyle Lounge' 'Crime Chronicles'
 'Detective Diaries' 'Life Lessons' 'Current Affairs' 'Finance Focus'
 'Laugh Line' 'True Crime Stories' 'Business Insights' 'Fashion Forward'
 'Tune Time'] None


In [244]:
pubday_onehot = pd.get_dummies(train_c[['Podcast_Name', 'Publication_Day']])
pubday_onehot.head()

,Podcast_Name_Athlete's Arena,Podcast_Name_Brain Boost,Podcast_Name_Business Briefs,Podcast_Name_Business Insights,Podcast_Name_Comedy Corner,Podcast_Name_Crime Chronicles,Podcast_Name_Criminal Minds,Podcast_Name_Current Affairs,Podcast_Name_Daily Digest,Podcast_Name_Detective Diaries,...,Podcast_Name_Tune Time,Podcast_Name_Wellness Wave,Podcast_Name_World Watch,Publication_Day_Friday,Publication_Day_Monday,Publication_Day_Saturday,Publication_Day_Sunday,Publication_Day_Thursday,Publication_Day_Tuesday,Publication_Day_Wednesday
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [245]:
train_c.drop(columns='Publication_Day', inplace=True)

In [246]:
train_c.drop(columns='Podcast_Name', inplace=True)

In [247]:
train_c = pd.concat([train_c, pubday_onehot], axis=1)

In [248]:
train_c.head()

,Episode_Title,Publication_Time,Episode_Sentiment,Business,Comedy,Education,Health,Lifestyle,Music,News,...,Podcast_Name_Tune Time,Podcast_Name_Wellness Wave,Podcast_Name_World Watch,Publication_Day_Friday,Publication_Day_Monday,Publication_Day_Saturday,Publication_Day_Sunday,Publication_Day_Thursday,Publication_Day_Tuesday,Publication_Day_Wednesday
0,98,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,26,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,16,2,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,45,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,86,1,-1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


---

In [261]:
import sklearn.linear_model as skl
from sklearn.ensemble import HistGradientBoostingRegressor as gbr

In [250]:
train_n.head()

,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads
0,NaN,74.81,NaN,0.0
1,119.80,66.95,75.95,2.0
2,73.90,69.97,8.97,0.0
3,67.17,57.22,78.70,2.0
4,110.51,80.07,58.68,3.0


In [251]:
train_c.head()

,Episode_Title,Publication_Time,Episode_Sentiment,Business,Comedy,Education,Health,Lifestyle,Music,News,...,Podcast_Name_Tune Time,Podcast_Name_Wellness Wave,Podcast_Name_World Watch,Publication_Day_Friday,Publication_Day_Monday,Publication_Day_Saturday,Publication_Day_Sunday,Publication_Day_Thursday,Publication_Day_Tuesday,Publication_Day_Wednesday
0,98,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,26,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,16,2,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,45,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,86,1,-1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [252]:
x = pd.concat([train_c, train_n], axis=1)

In [253]:
x.head()

,Episode_Title,Publication_Time,Episode_Sentiment,Business,Comedy,Education,Health,Lifestyle,Music,News,...,Publication_Day_Monday,Publication_Day_Saturday,Publication_Day_Sunday,Publication_Day_Thursday,Publication_Day_Tuesday,Publication_Day_Wednesday,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads
0,98,3,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,NaN,74.81,NaN,0.0
1,26,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,119.80,66.95,75.95,2.0
2,16,2,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,73.90,69.97,8.97,0.0
3,45,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,67.17,57.22,78.70,2.0
4,86,1,-1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,110.51,80.07,58.68,3.0


In [254]:
y = train.iloc[:,10]

In [225]:
y.head()

0    31.41998
1    88.01241
2    44.92531
3    46.27824
4    75.61031
Name: Listening_Time_minutes, dtype: float64

In [ ]:
reg = gbr().fit(x, y)

In [263]:
reg.score(x,y)

0.7697607138432372

# Evaluation

In [453]:
#from sklearn.metrics import mean_squared_error
#mse = mean_squared_error(actual, predicted)